In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [3]:
t1=time.time()
G=build_graph(cursor)
t2=time.time()
print(t2-t1)

0.6780967712402344


In [4]:
t1=time.time()
cmd="SELECT osm_id,contour_line_id,ST_asText(geometry) AS geometry FROM %s WHERE contour_line_id IS NOT NULL"%osm_nodes_table_name
cursor.execute(cmd)
nodes_data=pd.DataFrame(cursor.fetchall())
nodes_data['geometry']=nodes_data['geometry'].apply(lambda pt:loads(pt))
nodes_data=nodes_data.set_index('osm_id',drop=False)
nodes_data=gpd.GeoDataFrame(nodes_data,geometry='geometry',crs='epsg:4326')
nodes_data['old_geometry']=nodes_data['geometry'].copy()
crs=nodes_data.estimate_utm_crs()
nodes_data=nodes_data.to_crs(crs)
nodes_data['depth']=nodes_data['contour_line_id'].apply(lambda contour_line_id:G.nodes()[contour_line_id]['depth'])
t2=time.time()
print(t2-t1)

1.5086908340454102


In [5]:
contour_line_ids=nodes_data[nodes_data.depth>400]['contour_line_id'].unique()
contour_line_ids=[contour_line_id for contour_line_id in contour_line_ids if len(list(G.neighbors(contour_line_id)))>1]
contour_line_ids

[21765,
 115880,
 54878,
 4643,
 4538,
 5305,
 115887,
 51572,
 115988,
 24110,
 116089,
 56085,
 54393,
 1380,
 51038,
 13742,
 115908,
 8474,
 115873,
 115882,
 11416,
 54588,
 53737,
 115891]

In [19]:
root_contour_line_id

24110

In [ ]:
t1=time.time()
contour_line_ids=nodes_data['contour_line_id'].unique()
for k,root_contour_line_id in enumerate(contour_line_ids):
    if (k+1)%10==0:
        print('%i/%i'%(k+1,len(contour_line_ids)))
    t3=time.time()
    neighbor_contour_line_ids=list(G.neighbors(root_contour_line_id))
    print('%i neighbors'%len(neighbor_contour_line_ids))
    local_contours_line_ids=[root_contour_line_id]+neighbor_contour_line_ids
    local_contour_data=get_nodes_data(cursor,local_contours_line_ids)
    local_contour_data=local_contour_data.to_crs(crs)
    local_contour_data=local_contour_data.set_index('id')
    t4=time.time()
    print('setting up df took %f'%(t4-t3))
    local_nodes_data=nodes_data[nodes_data.contour_line_id==root_contour_line_id].copy()
    local_nodes_data=select_affine_basis_for_data_frame(local_nodes_data,local_contour_data)
    t5=time.time()
    print('finding affine basis took %f'%(t5-t4))
t2=time.time()
print(t2-t1)

184 neighbors
setting up df took 0.905825


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 77.642824
168 neighbors
setting up df took 1.012918


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 65.331997
3 neighbors
setting up df took 0.027647
finding affine basis took 0.031157
4 neighbors
setting up df took 0.017521
finding affine basis took 0.018185
2 neighbors
setting up df took 0.005539
finding affine basis took 0.004379
174 neighbors
setting up df took 0.959767


...

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 53.643404
157 neighbors
setting up df took 1.091828


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 33.150312
1 neighbors
setting up df took 0.007109
finding affine basis took 0.004687
5 neighbors
setting up df took 0.025452
finding affine basis took 0.005362
1 neighbors
setting up df took 0.006801
finding affine basis took 0.003305
2 neighbors
setting up df took 0.008132
finding affine basis took 0.004137
20/4337
141 neighbors
setting up df took 1.006309


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
